In [1]:
import pandas as pd

melb_df = pd.read_csv('data/melb_data_ps.csv', sep = ',')

In [2]:
melb_df['Address'].nunique()

13378

In [3]:
## Извлечём из признака адреса характеристику подтипа улицы (улица, шоссе, авеню, бульвар)
# На вход данной функции поступает строка с адресом.
def get_street_type(address):
    # Создаём список географических пометок exclude_list.
    exclude_list = ['N', 'S', 'W', 'E']
    # Метод split() разбивает строку на слова по пробелу.
    # В результате получаем список слов в строке и заносим его в переменную address_list.
    address_list = address.split(' ')
    # Обрезаем список, оставляя в нём только последний элемент,
    # потенциальный подтип улицы, и заносим в переменную street_type. 
    street_type = address_list[-1]
    # Делаем проверку на то, что полученный подтип является географической пометкой.
    # Для этого проверяем его на наличие в списке exclude_list. 
    if street_type in exclude_list:
    # Если переменная street_type является географической пометкой,
    # переопределяем её на второй элемент с конца списка address_list.
        return address_list[-2]
    return street_type

In [4]:
street_types = melb_df['Address'].apply(get_street_type)
melb_df['StreetTypes'] = street_types
melb_df['StreetTypes'].nunique()

56

In [5]:
def eliminate_longs(address):
    small_list = []
    for a in melb_df['StreetTypes'].unique():
        if len(a) <= 3:
            small_list.append(a)
    for b in small_list:
        if set(b).issubset(set(address)):
            return b
    return address

melb_df['StreetTypes'] = melb_df['StreetTypes'].apply(eliminate_longs)

In [6]:
display(melb_df['StreetTypes'].nunique())
display(melb_df['StreetTypes'].value_counts())

36

St          8020
Rd          2827
Ct           648
Dr           447
Av           362
Gr           323
Pde          226
Pl           169
Cr           156
Cl           100
La            67
Bvd           66
Tce           48
Wy            40
Hwy           25
Sq            11
Esp            8
Cir            7
Gdns           4
Fairway        4
Mews           4
Righi          3
Victoria       2
Glade          1
Nook           1
Qy             1
Cove           1
Res            1
Loop           1
East           1
Dell           1
Eyrie          1
Athol          1
Hts            1
Outlook        1
Woodland       1
Name: StreetTypes, dtype: int64

In [7]:
popular_stypes = street_types.value_counts().nlargest(10).index
display(popular_stypes)

Index(['St', 'Rd', 'Ct', 'Dr', 'Av', 'Gr', 'Pde', 'Pl', 'Cr', 'Cl'], dtype='object')

In [8]:
melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other')
melb_df['StreetType'].nunique()

11

In [9]:
melb_df.drop('Address', axis=1, inplace=True)

In [10]:
melb_df['WeekdaySale'] = pd.to_datetime(melb_df['Date'], yearfirst = True)
melb_df['WeekdaySale'] = melb_df['WeekdaySale'].dt.day_of_week
get_weekend = lambda x: 1 if x == 5 or x == 6 else 0
melb_df['Weekend'] = melb_df['WeekdaySale'].apply(get_weekend)
melb_df['Weekend']

C:\Users\qwert\AppData\Local\Temp\ipykernel_10212\1246957274.py:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  melb_df['WeekdaySale'] = pd.to_datetime(melb_df['Date'], yearfirst = True)


0        1
1        1
2        0
3        0
4        0
        ..
13575    1
13576    1
13577    1
13578    1
13579    1
Name: Weekend, Length: 13580, dtype: int64

In [11]:
top_sellers = melb_df['SellerG'].value_counts().nlargest(49).index
melb_df['SellerG'] = melb_df['SellerG'].apply(lambda x: x if x in top_sellers else 'other')
result = melb_df[melb_df['SellerG'] == 'Nelson']['Price'].min() / melb_df[melb_df['SellerG'] == 'other']['Price'].min()
round(result, 1)

1.3

In [12]:
# Давайте определим число уникальных категорий в каждом столбце нашей таблицы melb_df.

# создаём пустой список
unique_list = []
# пробегаемся по именам столбцов в таблице
for col in melb_df.columns:
    # создаём кортеж (имя столбца, число уникальных значений)
    item = (col, melb_df[col].nunique(), melb_df[col].dtypes)
    # добавляем кортеж в список
    unique_list.append(item)
# создаём вспомогательную таблицу и сортируем её
unique_counts = pd.DataFrame(unique_list, columns = ['Column_name', 'Num_Unique', 'Type']).sort_values(by='Num_Unique', ignore_index=True)
unique_counts

,Column_name,Num_Unique,Type
0,Weekend,2,int64
1,Type,3,object
2,Method,5,object
3,WeekdaySale,7,int64
4,Regionname,8,object
5,Rooms,9,int64
6,Bathroom,9,int64
7,StreetType,11,object
8,Car,11,int64
9,Bedroom,12,int64


In [13]:
# Начинаем изучать тип "Категории"

melb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          13580 non-null  int64  
 1   Suburb         13580 non-null  object 
 2   Rooms          13580 non-null  int64  
 3   Type           13580 non-null  object 
 4   Price          13580 non-null  float64
 5   Method         13580 non-null  object 
 6   SellerG        13580 non-null  object 
 7   Date           13580 non-null  object 
 8   Distance       13580 non-null  float64
 9   Postcode       13580 non-null  int64  
 10  Bedroom        13580 non-null  int64  
 11  Bathroom       13580 non-null  int64  
 12  Car            13580 non-null  int64  
 13  Landsize       13580 non-null  float64
 14  BuildingArea   13580 non-null  float64
 15  YearBuilt      13580 non-null  int64  
 16  CouncilArea    12211 non-null  object 
 17  Lattitude      13580 non-null  float64
 18  Longti

In [14]:
# Сделаем преобразование столбцов к типу данных category

cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_df.columns: # цикл по именам столбцов
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: # проверяем условие
        melb_df[col] = melb_df[col].astype('category') # преобразуем тип столбца
melb_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   index          13580 non-null  int64   
 1   Suburb         13580 non-null  object  
 2   Rooms          13580 non-null  int64   
 3   Type           13580 non-null  category
 4   Price          13580 non-null  float64 
 5   Method         13580 non-null  category
 6   SellerG        13580 non-null  category
 7   Date           13580 non-null  object  
 8   Distance       13580 non-null  float64 
 9   Postcode       13580 non-null  int64   
 10  Bedroom        13580 non-null  int64   
 11  Bathroom       13580 non-null  int64   
 12  Car            13580 non-null  int64   
 13  Landsize       13580 non-null  float64 
 14  BuildingArea   13580 non-null  float64 
 15  YearBuilt      13580 non-null  category
 16  CouncilArea    12211 non-null  category
 17  Lattitude      13580 non-null  

In [15]:
melb_df['Regionname'].cat.categories
melb_df['Regionname'].cat.codes

0        2
1        2
2        2
3        2
4        2
        ..
13575    4
13576    6
13577    6
13578    6
13579    6
Length: 13580, dtype: int8

In [16]:
melb_df['Type'] = melb_df['Type'].cat.rename_categories({
    'u': 'unit',
    't': 'townhouse',
    'h': 'house'
})
melb_df['Type']

0        house
1        house
2        house
3        house
4        house
         ...  
13575    house
13576    house
13577    house
13578    house
13579    house
Name: Type, Length: 13580, dtype: category
Categories (3, object): ['house', 'townhouse', 'unit']

In [17]:
new_houses_types = pd.Series(['unit', 'house', 'flat', 'flat', 'house'])
new_houses_types = new_houses_types.astype(melb_df['Type'].dtype)
display(new_houses_types)

0     unit
1    house
2      NaN
3      NaN
4    house
dtype: category
Categories (3, object): ['house', 'townhouse', 'unit']

In [18]:
melb_df['Type'] = melb_df['Type'].cat.add_categories('flat')
new_houses_types = pd.Series(['unit', 'house', 'flat', 'flat', 'house'])
new_houses_types = new_houses_types.astype(melb_df['Type'].dtype)
new_houses_types

0     unit
1    house
2     flat
3     flat
4    house
dtype: category
Categories (4, object): ['house', 'townhouse', 'unit', 'flat']